# Missing value imputation using Sciket Learn - Pipeline 

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
print(train.shape , test.shape)

(1460, 81) (1459, 80)


In [3]:
X_train = train.drop(columns='SalePrice' , axis = 'columns')
y_train = train['SalePrice']
X_test = test.copy()
print(X_train.shape , y_train.shape , X_test.shape)

(1460, 80) (1460,) (1459, 80)


# Imputing missing value 

In [4]:
isnull_sum = X_train.isnull().sum()
isnull_sum

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 80, dtype: int64

In [5]:
# finding the numerical variables with missing value
num_vars = X_train.select_dtypes(include=['int64' , 'float64']).columns
num_vars_miss = [var for var in num_vars if isnull_sum[var] > 0]
num_vars_miss

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [6]:
# finding the categorical variables with missing value
cat_vars = X_train.select_dtypes(include=['object']).columns
cat_vars_miss = [var for var in cat_vars if isnull_sum[var] > 0]
cat_vars_miss

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [7]:
# numerical variables
num_var_mean = ['LotFrontage']  # fill mean in 'LotFrontage'
num_var_median = ['MasVnrArea','GarageYrBlt'] # fill median in 'MasVnrArea','GarageYrBlt'

# categorical variables
cat_vars_mode = ['Alley','MasVnrType','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical','FireplaceQu']
cat_vars_missing = ['GarageType','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']

In [8]:
# Creating Pipline for different strategy(mean,median,mode,constant(any value of our choice))

# 'mean' strategy
num_var_mean_imputer = Pipeline(steps = [('imputer', SimpleImputer(strategy='mean'))])

# 'median' strategy
num_var_median_imputer = Pipeline(steps = [('imputer', SimpleImputer(strategy='median'))])

# 'mode' strategy
cat_vars_mode_imputer = Pipeline(steps = [('imputer', SimpleImputer(strategy='most_frequent'))])

# 'missing(constant)' strategy
cat_vars_missing_imputer = Pipeline(steps = [('imputer', SimpleImputer(strategy='constant',fill_value='missing'))])

In [9]:
# apply on the columns(any name, Pipeline for different strategy  , columns(onto  which we want to apply it))
preprocessor = ColumnTransformer(transformers=[
                                ('mean_imputer' , num_var_mean_imputer , num_var_mean) ,
                                ('median_imputer' , num_var_median_imputer , num_var_median) ,
                                ('mode_imputer' , cat_vars_mode_imputer , cat_vars_mode),
                                ('missing_imputer' ,cat_vars_missing_imputer ,cat_vars_missing)
                                              ])

In [10]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('missing_imputer',
                                 Pipe

In [11]:
preprocessor.transform

<bound method ColumnTransformer.transform of ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('missing_im

In [12]:
# to check the mean,median ,mode values found by the 'fit' method
preprocessor.named_transformers_['mean_imputer'].named_steps['imputer'].statistics_
# this is the mean value for 'LotFrontage'

array([70.04995837])

In [13]:
train['LotFrontage'].mean() #both are same 

70.04995836802665

In [14]:
preprocessor.named_transformers_['mode_imputer'].named_steps['imputer'].statistics_

array(['Grvl', 'None', 'TA', 'TA', 'No', 'Unf', 'Unf', 'SBrkr', 'Gd'],
      dtype=object)

In [15]:
# apply on the X_train and X_test dataset
X_train_clean = preprocessor.transform(X_train)
X_test_clean = preprocessor.transform(X_test)

In [16]:
X_train_clean

array([[65.0, 196.0, 2003.0, ..., 'missing', 'missing', 'missing'],
       [80.0, 0.0, 1976.0, ..., 'missing', 'missing', 'missing'],
       [68.0, 162.0, 2001.0, ..., 'missing', 'missing', 'missing'],
       ...,
       [66.0, 0.0, 1941.0, ..., 'missing', 'GdPrv', 'Shed'],
       [68.0, 0.0, 1950.0, ..., 'missing', 'missing', 'missing'],
       [75.0, 0.0, 1965.0, ..., 'missing', 'missing', 'missing']],
      dtype=object)

In [17]:
preprocessor.transformers_

[('mean_imputer',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['LotFrontage']),
 ('median_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['MasVnrArea', 'GarageYrBlt']),
 ('mode_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu']),
 ('missing_imputer',
  Pipeline(steps=[('imputer',
                   SimpleImputer(fill_value='missing', strategy='constant'))]),
  ['GarageType',
   'GarageFinish',
   'GarageQual',
   'GarageCond',
   'PoolQC',
   'Fence',
   'MiscFeature']),
 ('remainder',
  'drop',
  [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   

In [18]:
X_train_clean_miss_var = pd.DataFrame(X_train_clean , columns=num_var_mean+num_var_median+cat_vars_mode+cat_vars_missing)
X_train_clean_miss_var.head()

,LotFrontage,MasVnrArea,GarageYrBlt,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,196.0,2003.0,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,missing,missing,missing
1,80.0,0.0,1976.0,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
2,68.0,162.0,2001.0,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
3,60.0,0.0,1998.0,Grvl,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,missing,missing,missing
4,84.0,350.0,2000.0,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing


In [19]:
X_train_clean_miss_var.isnull().sum().sum()

0

In [20]:
train['Alley'].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [21]:
X_train_clean_miss_var['Alley'].value_counts()
# as Grvl was the mode in train dataset so this has been fill and therefore it has increased here

Grvl    1419
Pave      41
Name: Alley, dtype: int64

In [22]:
X_train_clean_miss_var['MiscFeature'].value_counts()

missing    1406
Shed         49
Gar2          2
Othr          2
TenC          1
Name: MiscFeature, dtype: int64